# CIFAR-10 Dataset

The CIFAR-10 dataset consists of __60000 32x32__ colour images in __10 classes__, with 6000 images per class. There are 50000 training images and 10000 test images.

# Prep Data

In [ ]:
import os
import urllib.request
import hashlib
import tarfile
import pickle
import numpy as np
import gzip

Download approximately 170MB of data.

In [ ]:
url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
fname = 'cifar-10-python.tar.gz'

if not os.path.isfile(fname):
    urllib.request.urlretrieve(url, fname)

fsum = hashlib.md5(open(fname, 'rb').read()).hexdigest()
assert fsum == 'c58f30108f718f92721af3b95e74349a'

Load dataset

In [ ]:
def extract(tfile, member):
    buff = tfile.extractfile(member)
    pdata = pickle.load(buff, encoding='latin1')
    features = pdata['data']
    features = features.reshape([len(pdata['data']), 3, 32, 32])
    features = features.transpose(0, 2, 3, 1)
    labels = pdata['labels']
    return features, labels

with tarfile.open(fname, 'r:gz') as tfile:
    members = tfile.getmembers()
    x1, y1 = extract(tfile, 'cifar-10-batches-py/data_batch_1')
    x2, y2 = extract(tfile, 'cifar-10-batches-py/data_batch_2')
    x3, y3 = extract(tfile, 'cifar-10-batches-py/data_batch_3')
    x4, y4 = extract(tfile, 'cifar-10-batches-py/data_batch_4')
    x5, y5 = extract(tfile, 'cifar-10-batches-py/data_batch_5')    
    
    train_x_raw = np.concatenate([x1, x2, x3, x4, x5])
    train_y_raw = np.concatenate([y1, y2, y3, y4, y5])
    
    test_x_raw, test_y_raw = extract(tfile, 'cifar-10-batches-py/test_batch')
    
    del x1, x2, x3, x4, x5, y1, y2, y3, y4, y5

### Preprocess

Scale

In [ ]:
train_x.std()

In [ ]:
train_x.mean()

In [ ]:
train_x_mean = train_x.mean()

In [ ]:
train_x_std = train_x.std()

In [ ]:
train_x = (train_x - train_x_mean) / train_x_std
test_x = (test_x - train_x_mean) / train_x_std

In [ ]:
train_x = train_x_raw / 255
test_x = test_x_raw / 255

Dummies

In [ ]:
def onehot(data_0123):
    N = len(data_0123)
    res = np.zeros([N, 10])
    res[range(N), data_0123] = 1
    return res

train_y = onehot(train_y_raw)
test_y = onehot(test_y_raw)

Split - dataset is already shuffled

In [ ]:
idx = int(len(train_x)*0.1)

valid_x = train_x[-idx:]
valid_y = train_y[-idx:]

train_x = train_x[:-idx]
train_y = train_y[:-idx]

### Save to drive

In [ ]:
#raise  # Sentinel
with gzip.open('CIFAR-10.gz', 'wb') as gfile:
    pickle_dict = {'train_x': train_x, 'train_y': train_y,
                   'valid_x': valid_x, 'valid_y': valid_y,
                   'test_x': test_x, 'test_y': test_y }
    pickle.dump(pickle_dict, gfile)

# Checkpoint

In [ ]:
import gzip
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tables

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
with tf.Session(config=config) as sess:
    devs = sess.list_devices()
    print('\n'.join([x.name for x in devs]))

In [ ]:
with gzip.open('CIFAR-10.gz', 'rb') as gfile:
    pickle_dict = pickle.load(gfile)
    train_x = pickle_dict['train_x']
    train_y = pickle_dict['train_y']
    valid_x = pickle_dict['valid_x']
    valid_y = pickle_dict['valid_y']
    test_x = pickle_dict['test_x']
    test_y = pickle_dict['test_y']
    del pickle_dict  # free up memory

# Explore Data

In [ ]:
lab2txt = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

idx = 43
plt.imshow(train_x[idx])
print(lab2txt[np.argmax(train_y[idx])])

# Model

### High-Level Model

In [ ]:
tf.reset_default_graph()

xi = tf.placeholder(name='xi', shape=[None, 32, 32, 3], dtype=tf.float32)
yi = tf.placeholder(name='yi', shape=[None, 10], dtype=tf.float32)
kp = tf.placeholder(name='kp', shape=[], dtype=tf.float32)

model = tf.layers.conv2d(xi,
    filters=32, kernel_size=[5, 5], strides=[1, 1], padding='SAME', activation=tf.nn.relu, name='Conv_1')
model = tf.layers.max_pooling2d(model,
    pool_size=[2, 2], strides=[2, 2], padding='SAME', name='Pool_1')

model = tf.layers.conv2d(model,
    filters=64, kernel_size=[5, 5], strides=[1, 1], padding='SAME', activation=tf.nn.relu, name='Conv_2')
model = tf.layers.max_pooling2d(model,
    pool_size=[2, 2], strides=[2, 2], padding='SAME', name='Pool_2')

model = tf.layers.conv2d(model,
    filters=128, kernel_size=[5, 5], strides=[1, 1], padding='SAME', activation=tf.nn.relu, name='Conv_3')
model = tf.layers.max_pooling2d(model,
    pool_size=[2, 2], strides=[2, 2], padding='SAME', name='Pool_3')

model = tf.layers.flatten(model)
model = tf.layers.dense(model, 128, activation=tf.nn.relu, name='Dense_1')
model = tf.nn.dropout(model, kp)
logits = tf.layers.dense(model, 10, activation=None, name='Logits')

probabilities = tf.nn.softmax(logits)

loss = tf.losses.softmax_cross_entropy(yi, logits)

optimizer = tf.train.AdamOptimizer()
grads_and_vars = optimizer.compute_gradients(loss)
train_op = optimizer.apply_gradients(grads_and_vars)

is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(yi, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

graph = tf.get_default_graph()

### Investigate graph

In [ ]:
# Display all tensor names in graph
[n.name for n in tf.get_default_graph().as_graph_def().node]

In [ ]:
# Display all trainable variables
tf.trainable_variables()

In [ ]:
# Display gradient tensors
grads_and_vars

In [ ]:
# Save tensorboard graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logdir='tf_log', graph=sess.graph)
    writer.flush()

### Build logging dict

In [ ]:
batch_save = 5

In [ ]:
dict_inout = {
    'batch_x' : xi[0:batch_save,:,:,:],
    'batch_y' : yi[0:batch_save,:],
    'probabilities' : probabilities[0:batch_save,:]
}

In [ ]:
dict_conv_1 = {
    'W': graph.get_tensor_by_name('Conv_1/kernel:0'),
    'b': graph.get_tensor_by_name('Conv_1/bias:0'),
    'dW': graph.get_tensor_by_name('gradients/Conv_1/Conv2D_grad/tuple/control_dependency_1:0'),
    'db': graph.get_tensor_by_name('gradients/Conv_1/BiasAdd_grad/tuple/control_dependency_1:0'),
    'z': graph.get_tensor_by_name('Conv_1/BiasAdd:0')[0:batch_save,:,:,:],
}

In [ ]:
dict_conv_2 = {
    'W': graph.get_tensor_by_name('Conv_2/kernel:0'),
    'b': graph.get_tensor_by_name('Conv_2/bias:0'),
    'dW': graph.get_tensor_by_name('gradients/Conv_2/Conv2D_grad/tuple/control_dependency_1:0'),
    'db': graph.get_tensor_by_name('gradients/Conv_2/BiasAdd_grad/tuple/control_dependency_1:0'),
    'z': graph.get_tensor_by_name('Conv_2/BiasAdd:0')[0:batch_save,:,:,:],
}

In [ ]:
dict_conv_3 = {
    'W': graph.get_tensor_by_name('Conv_3/kernel:0'),
    'b': graph.get_tensor_by_name('Conv_3/bias:0'),
    'dW': graph.get_tensor_by_name('gradients/Conv_3/Conv2D_grad/tuple/control_dependency_1:0'),
    'db': graph.get_tensor_by_name('gradients/Conv_3/BiasAdd_grad/tuple/control_dependency_1:0'),
    'z': graph.get_tensor_by_name('Conv_3/BiasAdd:0')[0:batch_save,:,:,:],
}

In [ ]:
dict_dense_1 = {
    'W': graph.get_tensor_by_name('Dense_1/kernel:0'),
    'b': graph.get_tensor_by_name('Dense_1/bias:0'),
    'dW': graph.get_tensor_by_name('gradients/Dense_1/MatMul_grad/tuple/control_dependency_1:0'),
    'db': graph.get_tensor_by_name('gradients/Dense_1/BiasAdd_grad/tuple/control_dependency_1:0'),
    'z': graph.get_tensor_by_name('Dense_1/BiasAdd:0')[0:batch_save,:],
}

In [ ]:
dict_logits = {
    'W': graph.get_tensor_by_name('Logits/kernel:0'),
    'b': graph.get_tensor_by_name('Logits/bias:0'),
    'dW': graph.get_tensor_by_name('gradients/Logits/MatMul_grad/tuple/control_dependency_1:0'),
    'db': graph.get_tensor_by_name('gradients/Logits/BiasAdd_grad/tuple/control_dependency_1:0'),
    'z': graph.get_tensor_by_name('Logits/BiasAdd:0')[0:batch_save,:],
}

dict_metrics = {
    'loss': loss,
    'accuracy': accuracy,
}

In [ ]:
dict_layers = {
    'inout': dict_inout,
    'conv_1': dict_conv_1,
    'conv_2': dict_conv_2,
    'conv_3': dict_conv_3,
    'dense_1': dict_dense_1,
    'logits': dict_logits,
    'metrics': dict_metrics,
}

### Prep log file

In [ ]:
import tables_logger
import importlib
importlib.reload(tables_logger)
filename = 'outarray.h5'

In [ ]:
tables_logger.create_log(filename, dict_layers, batch_save)  # this overrides any existing file with empty lotc

In [ ]:
tables_logger.print_raw(filename)

In [ ]:
tables_logger.print_log(filename)

### Train Loop

Hyperparameters

In [ ]:
epochs = 1 # 5 # 40
batch_size = 250
keep_probability = 0.75

Train Loop

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        for batch_i, idx in enumerate(range(0, len(train_x), batch_size)):
            
            xb = train_x[idx:idx+batch_size]
            yb = train_y[idx:idx+batch_size]
            
            summary, _, _ = sess.run([dict_layers, train_op, loss], feed_dict={xi: xb, yi: yb, kp: keep_probability})
            
            tables_logger.append_log(filename, summary)
            
        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
        train_cost, train_acc = sess.run([loss, accuracy], feed_dict={xi: xb, yi: yb, kp: 1.0})
        valid_cost, valid_acc = sess.run([loss, accuracy], feed_dict={xi: valid_x, yi: valid_y, kp: 1.0})
        print('tc, vc, tacc, vacc', train_cost, valid_cost, train_acc, valid_acc)
        
    saver.save(sess, './model.ckpt')

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './model.ckpt')
    
    train_cost, train_acc = sess.run([loss, accuracy], feed_dict={xi: train_x[:10000], yi: train_y[:10000], kp: 1.0})
    valid_cost, valid_acc = sess.run([loss, accuracy], feed_dict={xi: valid_x, yi: valid_y, kp: 1.0})
    test_cost, test_acc   = sess.run([loss, accuracy], feed_dict={xi: test_x,  yi: test_y,  kp: 1.0})
    print('Train Set Accuracy:', train_acc)
    print('Validation Set Acc:', valid_acc)
    print('Test Set Accuracy: ', test_acc)